In [2]:
#==================================================================
#Program: blockReExtraction
#Version: 1.0
#Author: David Helminiak
#Date Created: September 7, 2024
#Date Last Modified: September 7, 2024
#Changelog: 1.0 - Assembly and cropping - September 2024
#Description: Extract best matching .tif block images for samples from the associated WSI .jpg files
#Operation: Move back into main program directory before running.
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#RNG seed value to ensure run-to-run consistency (-1 to disable)
manualSeedValue = 0

#Debugging mode
debugMode = False

#TQDM progress bar visualization flag
asciiFlag = False

#Load external libraries
exec(open("./CODE/EXTERNAL.py", encoding='utf-8').read())
from skimage.metrics import normalized_root_mse as compare_NRMSE

#When splitting WSI images, what size should the resulting blocks be (default: 400)
#Should remain consistent with block sizes given for training
blockSize = 400

#If visual confirmation that new block images match with a previous block set
visualConfirmation = False


In [84]:
#Store directory references
dir_data = '.' + os.path.sep + 'DATA' + os.path.sep
dir_blocks_data = dir_data + 'BLOCKS' + os.path.sep
dir_blocks_inputBlocks = dir_blocks_data + 'INPUT_BLOCKS' + os.path.sep
dir_blocks_inputWSI = dir_blocks_data + 'INPUT_WSI' + os.path.sep
dir_blocks_outputBlocks = dir_blocks_data + 'OUTPUT_BLOCKS' + os.path.sep
dir_blocks_outputVisuals = dir_blocks_data + 'OUTPUT_BLOCKS_VISUALS' + os.path.sep
file_blocks_labels = dir_blocks_inputBlocks + 'Patch_list.xlsx'

#Create storage locations for new block files and visuals
if os.path.exists(dir_blocks_outputBlocks): shutil.rmtree(dir_blocks_outputBlocks)
os.makedirs(dir_blocks_outputBlocks)
if os.path.exists(dir_blocks_outputVisuals): shutil.rmtree(dir_blocks_outputVisuals)
os.makedirs(dir_blocks_outputVisuals)

#Get list of all the previous blocks that are intended for comparison with new extractions
metadata = pd.read_excel(file_blocks_labels, header=None, names=['name', 'label'], converters={'name':str,'label':str})
blockNamesAll_blocks, blockLabelsAll_blocks = np.array(metadata['name']), np.array(metadata['label'])
blockSampleNamesAll_blocks = np.array([re.split('PS|_', blockName)[1] for blockName in blockNamesAll_blocks])
blockFilenamesAll_blocks = [dir_blocks_inputBlocks + 'S' + blockSampleNamesAll_blocks[blockIndex] + os.path.sep + blockNamesAll_blocks[blockIndex] + '.tif' for blockIndex in range(0, len(blockNamesAll_blocks))]


In [85]:
#Technically, only need to extract blocks for the samples that blockSourceVerification.ipynb identified as not having exact matches from the WSI
#sampleNames = ['10', '12', '13', '14', '15', '21', '3', '32', '36', '38', '40', '42', '43', '45', '48', '49', '52', '53', '55', '56', '59', '6', '63', '66', '69', '8']

#However, since adding row/column location data has to be done anyways, just go through all of them
sampleNames = np.unique(blockSampleNamesAll_blocks)

#Create subfolders for the block images derived from each sample
sampleFolders = [dir_blocks_outputBlocks + sampleName + os.path.sep for sampleName in sampleNames]
for sampleFolder in sampleFolders: os.makedirs(sampleFolder)
    
sampleVisualsFolders = [dir_blocks_outputVisuals + sampleName + os.path.sep for sampleName in sampleNames]
for sampleVisualsFolder in sampleVisualsFolders: os.makedirs(sampleVisualsFolder)


In [86]:
#Extract blocks for each sample
samplesToCheck = []
for sampleIndex, sampleName in tqdm(enumerate(sampleNames), total=len(sampleNames), desc='Samples', leave=True, ascii=asciiFlag):
    
    #Get filenames and load original blocks
    blockFilenames = np.asarray(blockFilenamesAll_blocks)[np.where(blockSampleNamesAll_blocks == sampleName)[0]]
    originalBlockImages = [cv2.cvtColor(cv2.imread(blockFilename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB) for blockFilename in blockFilenames]
    
    #Load the sample WSI
    imageWSI = cv2.cvtColor(cv2.imread(dir_blocks_inputWSI + sampleName + '.jpg', cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
    
    #Find the location of one of the blocks (assuming WSI was not resized before blocks were extracted)
    blockImage = cv2.cvtColor(cv2.imread(blockFilenames[0], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
    heat_map = cv2.matchTemplate(imageWSI, blockImage, cv2.TM_CCOEFF_NORMED)
    startRow, startColumn = np.unravel_index(np.argmax(heat_map), heat_map.shape)

    #Assuming that the blocks are non-overlapping and match blockSize expectations, extrapolate crop parameters applied to the WSI before splitting
    cropLeft, cropRight, cropTop, cropBottom = startColumn, startColumn, startRow, startRow
    while cropLeft > 0: cropLeft-=blockSize
    while cropRight < imageWSI.shape[1]: cropRight+=blockSize
    while cropTop > 0: cropTop-=blockSize
    while cropBottom < imageWSI.shape[0]: cropBottom+=blockSize
    cropLeft, cropRight, cropTop, cropBottom = cropLeft+blockSize, cropRight-blockSize, cropTop+blockSize, cropBottom-blockSize

    #Crop the original WSI, as was done to the orignal WSI before splitting
    imageWSI = imageWSI[cropTop:cropBottom, cropLeft:cropRight]
    
    #Split the cropped WSI into blocks and flatten, creating a matched indexed locations list
    numRows, numColumns = math.ceil(imageWSI.shape[0]/blockSize), math.ceil(imageWSI.shape[1]/blockSize)
    imageWSI = imageWSI.reshape(numRows, blockSize, numColumns, blockSize, imageWSI.shape[2]).swapaxes(1,2)
    imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])
    locations = [[rowNum, colNum] for rowNum in range(0, numRows) for colNum in range(0, numColumns)]
    
    #If the WSI were the same, then could split directly, but they aren't
    #Compute number of blocks
    #Crop the .jpg WSI, as was done to the orignal WSI before splitting
    #Split the cropped WSI into blocks and flatten, creating a matched indexed locations list
    #numRows, numColumns = int(np.floor(imageWSI.shape[0]/blockSize)), int(np.floor(imageWSI.shape[1]/blockSize))
    #imageWSI = imageWSI[:numRows*blockSize, :numColumns*blockSize]
    #imageWSI = imageWSI.reshape(numRows, blockSize, numColumns, blockSize, imageWSI.shape[2]).swapaxes(1,2)
    #imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])
    #locations = [[rowNum, colNum] for rowNum in range(0, numRows) for colNum in range(0, numColumns)]
    
    #Find best matching block, storing it, and a visual of the original/matched blocks if they do not match exactly
    for originalBlockIndex, originalBlockImage in tqdm(enumerate(originalBlockImages), total=len(originalBlockImages), desc='Blocks', leave=False, ascii=asciiFlag):
        #scores = [compare_SSIM(originalBlockImage, blockImage, data_range=255, channel_axis=-1) for blockImage in tqdm(imageWSI, desc='Scores', leave=False, ascii=asciiFlag)]
        scores = [compare_NRMSE(originalBlockImage, blockImage) for blockImage in tqdm(imageWSI, desc='Scores', leave=False, ascii=asciiFlag)]        
        bestIndex = np.argmin(scores)
        blockImage, location = imageWSI[bestIndex], locations[bestIndex]
        blockIndex = os.path.basename(blockFilenames[originalBlockIndex]).split('_')[-1].split('.tif')[0]
        filenameOutput = sampleFolders[sampleIndex] + 'PS' + sampleName + '_' + blockIndex + '_' + str(location[0]) + '_' + str(location[1]) + '.tif'
        writeSuccess = cv2.imwrite(filenameOutput, cv2.cvtColor(blockImage, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        if np.sum(originalBlockImage-blockImage) != 0: 
            samplesToCheck.append(sampleName)
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(6, 4))
            ax[0].imshow(blockImage)
            ax[1].imshow(originalBlockImage)
            ax[0].set_title('New')
            ax[1].set_title('Original')
            fig.suptitle('Sample ' + sampleName)
            plt.tight_layout()
            filenameOutput = sampleVisualsFolders[sampleIndex] + 'PS' + sampleName + '_' + blockIndex + '_' + str(location[0]) + '_' + str(location[1]) + '.tif'
            plt.savefig(filenameOutput)
            #plt.show()
            plt.close()

samplesToCheck = np.unique(samplesToCheck)


Samples:   0%|          | 0/2 [00:00<?, ?it/s]

Blocks:   0%|          | 0/466 [00:00<?, ?it/s]

Scores:   0%|          | 0/1080 [00:00<?, ?it/s]

Scores:   0%|          | 0/1080 [00:00<?, ?it/s]

Scores:   0%|          | 0/1080 [00:00<?, ?it/s]

...

Scores:   0%|          | 0/1716 [00:00<?, ?it/s]

Scores:   0%|          | 0/1716 [00:00<?, ?it/s]

Scores:   0%|          | 0/1716 [00:00<?, ?it/s]

Scores:   0%|          | 0/1716 [00:00<?, ?it/s]

Scores:   0%|          | 0/1716 [00:00<?, ?it/s]

In [ ]:

    
    
    #Extract new blocks from the WSI
    error = False
    blockIndex = 0
    for colNum in range(0, numColumns):
        startCol, endCol = colNum*blockSize, (colNum+1)*blockSize
        for rowNum in range(0, numRows): 
            startRow, endRow = rowNum*blockSize, (rowNum+1)*blockSize
            blockImage = imageWSI[startRow: endRow, startCol:endCol]
            areaPercentageR = np.sum(blockImage[:,:,0]>thresholdRedChannel)/blockArea
            if areaPercentageR > thresholdPercentage:
                blockIndex += 1
                filenameOutput = sampleFolders[sampleIndex] + 'PS' + sampleName + '_' + str(blockIndex) + '_' + str(rowNum) + '_' + str(colNum) + '.tif'
                writeSuccess = cv2.imwrite(filenameOutput, cv2.cvtColor(blockImage, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
                if visualConfirmation:
                    filenameCompare = blockFilenames[np.where(blockIndices==blockIndex)[0][0]]
                    compareBlockImage = cv2.cvtColor(cv2.imread(filenameCompare, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
                    if np.sum(np.mean(np.abs(blockImage-compareBlockImage), -1)) > 0:
                        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
                        ax[0].set_title('Original')
                        ax[0].imshow(compareBlockImage)
                        ax[1].set_title('New')
                        ax[1].imshow(blockImage)
                        fig.suptitle('Sample ' + sampleName)
                        plt.tight_layout()
                        plt.show()
                        plt.close()
                        